# TensorFlow 기본 구조 이해

In [1]:
import tensorflow as tf

print("TensorFlow version: {}".format(tf.VERSION))

TensorFlow version: 1.11.0


## TensorFlow Programming Stack

<img src="https://user-images.githubusercontent.com/11681225/46912209-324b8480-cfaa-11e8-9a30-6cd327841470.png" width="70%">

## Execution Style

* [Eager execution](https://www.tensorflow.org/get_started/eager)
  * 최근에 나온 방식
  * 변수값을 바로 확인할 수 있음
  * 마치 `numpy` 짜듯이 짤 수 있음
  * 향후 **TF version 2.0에 기본** 이 될 예정
* [Graph execution](https://www.tensorflow.org/get_started/get_started_for_beginners)
  * TensorFlow 초창기 구현 방법
  * 여전히 많이 이용하고 있음
  * 속도면에서 아직 `eager`모드보다 우위에 있음
  * **Graph 선언부분과 실행(Session)하는 부분이 분리** 되어 있음
    * Phase1: Graph 선언
    * Phase2: Session

### Phase1: Graph 생성

* 변수를 생성하고 계산을 한다는 것은 그래프를 만드는 과정
* `a`를 출력하면 **계산된 값**이 나오지 않음

<img width="60%" alt="tf_graph" src="https://user-images.githubusercontent.com/11681225/46912242-f95fdf80-cfaa-11e8-8e1c-00af08c281dc.png">

In [2]:
a = tf.add(3, 5)
print(a)

Tensor("Add:0", shape=(), dtype=int32)


* `a = 3 + 5`의 의미
* `a`를 출력하면 8이 나오는게 일반적인 프로그래밍 스타일
* TensorFlow는 위의 단계 (Graph를 만드는 과정) 에서는 값이 출력되지 않음
  * 출력하면 (name, shape, type) 의 정보가 나옴 (값이 아닌 것에 유의)
* Graph를 만든다는 의미는 TF 내부에서 실제로 연산을 밑의 그림처럼 구성한다는 뜻

<img width="40%" alt="tf_add1" src="https://user-images.githubusercontent.com/11681225/46912252-43e15c00-cfab-11e8-9488-bc5c6bccab0b.png">

### Phase2: Session 실행

* 위의 과정은 그래프 형태만 만들어놓음
* 실제 계산은 `tf.Session()`을 실행하여 계산함
* 마치 파이프에 물(데이터)을 흘려보내는 것과 비슷함
* `tf.Session()`을 열면 TF default로 GPU 메모리를 다 잡아버림
  * 그것을 방지하기 위해 `gpu_options`을 다음과 같이 준다
* GPU 사용량 확인하는 명령어
  * `nvidia-smi`
  * `watch`라는 명령어와 함께 쓰면 계속 갱신하면서 메모리 변화를 볼 수 있음
    * `watch -n 1 nvidia-smi`

In [3]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
#sess = tf.Session()
sess = tf.Session(config=sess_config)
print(sess.run(a))
sess.close()

8


<img width="40%" alt="tf_add2" src="https://user-images.githubusercontent.com/11681225/46912254-66737500-cfab-11e8-87e1-1d8364f5cc70.png">

### `tf.Session()`을 `with` 구문으로

* `session`을 열면 `sess.close()`로 명시적으로 닫아줘야 한다.
* `with` 구문이 끝나면 알아서 `session`이 닫힌다.

In [4]:
with tf.Session(config=sess_config) as sess:
  print(sess.run(a))

8


In [5]:
with tf.Session(config=sess_config) as sess:
  print(a.eval())

8


* `with tf.Session()` 구문 안에서는 `sess.run()`대신 `eval()`을 쓸 수 있다

### `tf.InteractiveSession()`

* `tf.InteractiveSession()`에서는 세션 객체를 계속 참조하지 않고 값을 볼 수 있다.

In [6]:
sess = tf.InteractiveSession(config=sess_config)
print(a.eval())

8


### 약간 더 복잡한 계산

In [7]:
x = 2
y = 3
w = tf.add(x, y)
z = tf.multiply(x, y)
p = tf.pow(z, w)
print(p)
with tf.Session(config=sess_config) as sess:
  print(sess.run(p))

Tensor("Pow:0", shape=(), dtype=int32)
7776


<img width="60%" alt="tf_more_calculation" src="https://user-images.githubusercontent.com/11681225/46912256-7db26280-cfab-11e8-903c-087f9a2688aa.png">